## Demonstrating a DataMined Economy

In this IPython notebook, we walk through construction of a DataMined economy. Recall the basic construction of DataMined: "tokens" are issued for addition of valid data to the network. Users who add data to the network are called "miners." Miners are typically assumed to be consumers who'd like to contribute and safeguard their individual data, however they may be non-profits or businesses as well. Data contributed in to the network is stored on storage nodes within the network. For small datasets, miners can store data locally on their own machines.

Note that since miners are rewarded for contributing data to the network via token issuance, there is a strong incentive to attack the network by contributing fake data. Consequently, network security is critically dependent on `Validator`s which can detect fraudulent data. Another interesting aspect of network design is that direct download of data off the network can't be permitted due to excessive risk of data leakage. As a result, computation must be performed on the network itself.

The "tokens" used in this network are called `DataCoin`s. These tokens conform to the [ERC20](https://theethereum.wiki/w/index.php/ERC20_Token_Standard) for tokens issued on the Ethereum network. For those new to ethereum development, the token is a form of currency whose fidelity is guaranteed by Ethereum network miners (note the distinction; ethereum miners are *not* necessarily DataMined data miners). Customers who wish to perform computation on data in the network must pay both miners and compute nodes in tokens to request that computation be performed on interesting data.

In this notebook, we introduce the core DataMined package `datamined` and demonstrate how to use `datamined` to construct an example data economy. Since this is an example economy, this code runs on a single local machine. In addition, tokens issued in this economy are run on a tiny, local instantiation of the Ethereum protocol. (The same code can be deployed onto larger Ethereum networks, but would take much longer to run to completion).

Let's start by defining the number of participants in our economy.

In [1]:
N_miners = 100
N_compute_nodes = 10
N_buyers = 5

In this economy, we have `100` data miners who will contribute their data to the network. Each of these miners should be rewarded for their trouble with `DataCoin` tokens. In order for miner data to be accepted however, miners must pass the gauntlet of the validator. The submodule `datamined.valid` provides implementations of validators for different data-types.

Let's see how miners join the network.

In [2]:
import datamined as dm
# TODO(rbharath): The eth_utils package emits many unsavory warnings. 
# I haven't yet been able to figure out a good way to suppress them.

miners = []
for n in range(N_miners):
  # Create a separate client wallet for each miner
  miner_wallet = dm.coins.LocalGethWallet()
  # TODO(rbharath): Modify example so that each miner has a separate private key
  miner_private_key = "ignored"
  # TODO(rbharath): The InsecureFileClient doesn't perform any at-rest encryption of data.
  # Swap out for a wallet that encrypts data that's on disk.
  miner = dm.data.InsecureFileClient(miner_private_key, miner_wallet)
  miners.append(miner)

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

...

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

Now that miners have been created, let's allow miners to load their data and receive tokens for their trouble. In this example, we will assume that miners are loading personal genomic data onto the network. If the validator accepts their genomic data, then the miner is rewarded with `DataCoin` tokens for their trouble.

In [3]:
import random

# This simple validator assumes that all provided data is valid.
# TODO(rbharath): Swap out this simple validator for a more 
validator = dm.valid.NaiveGenomicValidator()

# TODO(rbharath): The ledgers provide the location of the stored data. This might be an unsightly API
# in the long-run and should perhaps be refactored out.
ledgers = []
for n in range(N_miners):
  print("Adding data for miner ", n)
  miner = miners[n]
  miner_data = "".join([random.choice(["A", "C", "G", "T"]) for _ in range(100)])
  ledger = miner.store(miner_data, validator)
  coins_issued = miner.get_wallet().get_balance()
  print("  %d coins were issued" % coins_issued)
  assert coins_issued > 0
  ledgers.append(ledger)

Adding data for miner  0


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

  1 coins were issued
Adding data for miner  1


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_abi/abi.py:92: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded

  1 coins were issued
Adding data for miner  2


...

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_abi/abi.py:92: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3

  1 coins were issued
Adding data for miner  98
  1 coins were issued
Adding data for miner  99


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_abi/abi.py:92: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded

  1 coins were issued


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_abi/abi.py:92: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "


Now that miners have added data onto the data economy, let's see how to add compute nodes onto the mining economy. The submodule `datamined.compute` provides utilities for creating compute nodes and adding them onto the networ.

In [4]:
# At this point, there's currency floating around. This currency
# should incentivize the entrace of computational nodes onto the
# economy.
compute_nodes = []
for m in range(N_compute_nodes):
  print("Adding compute node ", m)
  # Create a node wallet
  node_wallet = dm.coins.LocalGethWallet()
  assert node_wallet.get_balance() == 0

  # TODO(rbharath): Add a more complex computation here.
  node = dm.compute.GenomicCountInsecureFileNode(node_wallet)
  compute_nodes.append(node)

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

Adding compute node  0


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

Adding compute node  1


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: De

...

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: Deprec

Adding compute node  8


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

Adding compute node  9


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/project.py:235: DeprecationWarning: project.contracts_source_dir has been replaced by the plural, project.contracts_source_dirs which is an iterable of all source directories populus will search for contracts.  Please update your code accordingly as this API will be removed in a future release
  "project.contracts_source_dir has been replaced by the plural, "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarn

Now that both data and compute powers are available on the network, let's see how to enable data customers to perform to perform computation on the network.

In [5]:
import numpy as np

# TODO(rbharath): There isn't currently a way to exchange ethereum for DataCoins directly.
# Consequently, only the miners have money on the network and can actually pay for computation.
# Fix this so external buyers can join the network.
data_customer = miners[0]

# Performing computaiton on the network.
# Select a random compute node
node = compute_nodes[np.random.randint(0, N_compute_nodes)]
# Select a random miner (who is not the data_customer)
data = miners[np.random.randint(1, N_miners)]
ledger = ledgers[np.random.randint(0, N_miners)]
# Obtain ledger key
ledger_key = data.get_ledger_key(ledger)

result = node.compute(ledger, ledger_key, "count_basepairs")
print("result: ", result)

# TODO(rbharath): The economy has a hole here where more money is paid than deducted! Plug the hole.
final_balance = node.get_wallet().get_balance()
print("Node now has %d DataCoins after payment" % final_balance)

/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which provides similar functionality through the `cytoolz.functoolz.compose` function.
  "The `compose` function has been deprecated and will be removed in a "
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_utils/functional.py:21: DeprecationWarning: The `compose` function has been deprecated and will be removed in a subsequent release of `ethereum-utils`.  It is recommended that you switch to use a library like `cytoolz` which pr

result:  {'A': 30, 'C': 29, 'G': 20, 'T': 21}
Node now has 4 DataCoins after payment


/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/populus/chain/tester.py:13: DeprecationWarning: currentProvider is deprecated in favor of `providers`, which is now a list
  self.rpc_methods = self.web3.currentProvider.rpc_methods
/home/rbharath/anaconda3/envs/datamined/lib/python3.6/site-packages/eth_abi/abi.py:92: DeprecationWarning: Automatic inference of hex encoded data has been deprecated. Please adjust your code to ensure that the data argument for `decode_single` is a byte string
  "Automatic inference of hex encoded data has been deprecated. "
